In [1]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema
from mlflow.types.schema import ColSpec

In [2]:
from sklearn.datasets import load_iris

In [23]:
data = load_iris(as_frame = True)

In [9]:
mlflow.set_experiment(experiment_name="mlflow-experiment")

2025/09/12 05:34:42 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///d:/Mlflow/mlruns/293285398803758555', creation_time=1757633682769, experiment_id='293285398803758555', last_update_time=1757633682769, lifecycle_stage='active', name='mlflow-experiment', tags={}>

In [11]:
col_specification=[
    ColSpec(type = "double", name=feature_name, required=True) for feature_name in data.feature_names
]

optional_column = [ColSpec(type = "double", name = "optional_column", required=False)]

model_input = Schema(inputs = col_specification + optional_column)

In [14]:
model_output = Schema(inputs = [ColSpec(type = "integer", name = 'species',required = True)])

In [15]:
model_signature = ModelSignature(inputs = model_input, outputs = model_output)

In [18]:
model_signature.to_dict()

{'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": true}, {"type": "double", "name": "sepal width (cm)", "required": true}, {"type": "double", "name": "petal length (cm)", "required": true}, {"type": "double", "name": "petal width (cm)", "required": true}, {"type": "double", "name": "optional_column", "required": false}]',
 'outputs': '[{"type": "integer", "name": "species", "required": true}]',
 'params': None}

In [ ]:
class CustomModel(mlflow.pyfunc.PythonModel):
    def predict(self, context, model_input):
        "more logic can be added here"
        return model_input

with mlflow.start_run(run_name = "custom-model-with-optional-inputs") as run:
    mlflow.pyfunc.log_model(
        artifact_path = "custom-model-with-optional-inputs",
        python_model = CustomModel(),
        signature = model_signature
    )

custom_model = mlflow.pyfunc.load_model(f"runs:/{run.info.run_id}/custom-model-with-optional-inputs")

c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\pyfunc\__init__.py:3242: UserWarning: An input example was not provided when logging the model. To ensure the model signature functions correctly, specify the `input_example` parameter. See https://mlflow.org/docs/latest/model/signatures.html#model-input-example for more details about the benefits of using input_example.
  color_warning(


In [31]:
data.data

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [38]:
X_with_optional_column = data.data.copy()
X_with_optional_column["optional_column"] = 100 * data.data["petal length (cm)"]
X_with_optional_column

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),optional_column
0,5.1,3.5,1.4,0.2,140.0
1,4.9,3.0,1.4,0.2,140.0
2,4.7,3.2,1.3,0.2,130.0
3,4.6,3.1,1.5,0.2,150.0
4,5.0,3.6,1.4,0.2,140.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,520.0
146,6.3,2.5,5.0,1.9,500.0
147,6.5,3.0,5.2,2.0,520.0
148,6.2,3.4,5.4,2.3,540.0


In [39]:
custom_model.predict(X_with_optional_column)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),optional_column
0,5.1,3.5,1.4,0.2,140.0
1,4.9,3.0,1.4,0.2,140.0
2,4.7,3.2,1.3,0.2,130.0
3,4.6,3.1,1.5,0.2,150.0
4,5.0,3.6,1.4,0.2,140.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,520.0
146,6.3,2.5,5.0,1.9,500.0
147,6.5,3.0,5.2,2.0,520.0
148,6.2,3.4,5.4,2.3,540.0


Infer Model Signature

In [40]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from mlflow.models.signature import infer_signature
from pprint import pprint

In [41]:
data = load_iris(as_frame = True)

X = data.data
y= data.target
y.name = "species"

model_signature = infer_signature(model_input=X, model_output=y)

pprint(model_signature.to_dict(),indent = 2)

{ 'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": '
            'true}, {"type": "double", "name": "sepal width (cm)", "required": '
            'true}, {"type": "double", "name": "petal length (cm)", '
            '"required": true}, {"type": "double", "name": "petal width (cm)", '
            '"required": true}]',
  'outputs': '[{"type": "integer", "name": "species", "required": true}]',
  'params': None}


c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [43]:
dtc = DecisionTreeClassifier()
dtc.fit(X,y)

with mlflow.start_run(run_name = "log_decision_tree_classifier") as run:
    mlflow.sklearn.log_model(
        sk_model = dtc,
        artifact_path = "model",
        signature = model_signature
    )

Optional Fields when working with infer_signature

In [45]:
infer_signature_with_optional_column = infer_signature(model_input = X_with_optional_column, model_output = y)
pprint(infer_signature_with_optional_column, indent =2)

inputs: 
  ['sepal length (cm)': double (required), 'sepal width (cm)': double (required), 'petal length (cm)': double (required), 'petal width (cm)': double (required), 'optional_column': double (required)]
outputs: 
  ['species': integer (required)]
params: 
  None



c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [47]:
X_with_optional_column["optional_column"] = None
print(X_with_optional_column) 

     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

    optional_column  
0    

In [49]:
infer_signature_with_optional_column = infer_signature(model_input = X_with_optional_column, model_output = y)

c:\Users\saqli\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [50]:
pprint(infer_signature_with_optional_column.to_dict(), indent = 2)

{ 'inputs': '[{"type": "double", "name": "sepal length (cm)", "required": '
            'true}, {"type": "double", "name": "sepal width (cm)", "required": '
            'true}, {"type": "double", "name": "petal length (cm)", '
            '"required": true}, {"type": "double", "name": "petal width (cm)", '
            '"required": true}, {"type": "any", "name": "optional_column", '
            '"required": false}]',
  'outputs': '[{"type": "integer", "name": "species", "required": true}]',
  'params': None}
